<a href="https://colab.research.google.com/github/KajetanFrackowiak/tensorflow_fundamentals/blob/main/06_transfer_learning_in_tensorflow_part_3_scaling_up_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning with TensorFlow Part 3: Scaling up

In [1]:
!nvidia-smi

Fri Dec 29 22:57:39 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 532.09                 Driver Version: 532.09       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650       WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   58C    P8                3W /  N/A|    160MiB /  4096MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
import zipfile

def unzip_data(filename):
    zip_ref =  zipfile.ZipFile(filename, "r")
    zip_ref.extractall()
    zip_ref.close()

In [4]:
import os

def walk_through_dir(dir_path):
    for dirpath, dirnames, filenames in os.walk(dir_path):
        print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'")

In [19]:
import datetime
import tensorflow as tf

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir
    )
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tensorboard_callback

In [5]:
!wget "https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip"
unzip_data("101_food_classes_10_percent.zip")

train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

--2023-12-30 10:19:54--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 2a00:1450:401b:800::201b, 2a00:1450:401b:80e::201b, 2a00:1450:401b:810::201b, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2a00:1450:401b:800::201b|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G  9.20MB/s    in 3m 1s   

2023-12-30 10:22:55 (8.59 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [6]:
walk_through_dir("101_food_classes_10_percent")

There are 2 directories and 0 images in '101_food_classes_10_percent'
There are 101 directories and 0 images in '101_food_classes_10_percent/test'
There are 0 directories and 250 images in '101_food_classes_10_percent/test/donuts'
There are 0 directories and 250 images in '101_food_classes_10_percent/test/sashimi'
There are 0 directories and 250 images in '101_food_classes_10_percent/test/macaroni_and_cheese'
There are 0 directories and 250 images in '101_food_classes_10_percent/test/bibimbap'
There are 0 directories and 250 images in '101_food_classes_10_percent/test/lobster_roll_sandwich'
There are 0 directories and 250 images in '101_food_classes_10_percent/test/baklava'
There are 0 directories and 250 images in '101_food_classes_10_percent/test/poutine'
There are 0 directories and 250 images in '101_food_classes_10_percent/test/bruschetta'
There are 0 directories and 250 images in '101_food_classes_10_percent/test/takoyaki'
There are 0 directories and 250 images in '101_food_classe

In [7]:
# Setup data input
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                               label_mode="categorical",
                                                                               image_size=IMG_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                shuffle=False)  # don't shuffle test data for prediction analysis

2023-12-30 12:08:49.796885: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-30 12:08:49.796944: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-30 12:08:49.905557: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-30 12:08:50.093831: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-30 12:08:51.518573: W tensorflow/compiler/tf2

Found 7575 files belonging to 101 classes.


2023-12-30 12:08:54.698796: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-30 12:08:55.132844: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Found 25250 files belonging to 101 classes.


## Train a big dog model with trainsfer learning on 10% of 101 food classes

* Create a ModelCHeckpint callback
* Create a data augmentation layer to build data augmentation right into the model
* Build a headless (no top layers( FUntional EfficientNetB0 backboned-model
* Compile our model
* Feature extract fo 5 full passes (5 epochs on the train dataset and validat on 15% of the test data, to save epoch time)

# Create checkpoint callback

In [25]:
checkpoint_path = "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True, # save only the model weights
                                                         monitor="val_accuracy", # save the model weights which score the best validation accuracy
                                                         save_best_only=True)

In [14]:
# Create data augmentation layer to incorporate it right into the model
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Setup data augmentation
data_augmentation = Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomZoom(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    # preprocessing.Rescaling(1/225.)  # rescale inputs of images to between 0 & 1, reuqired for models like ResNet50
], name="data_augmentation")

In [27]:
# Setup base model and freeze its layers (this will extract features)
base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False)
base_model.trainable = False

# Setup model architecture with trainable top layers
inputs = layers.Input(shape=(224, 224, 3), name="input_layer") # shape of input image
x = data_augmentation(inputs) # augment images (only happens during training)
x = base_model(x, training=False) # put the base model in inference mode so we can use it to extract features without updating the weights
x = layers.GlobalAveragePooling2D(name="global_average_pooling")(x) # pool the outputs of the base model
outputs = layers.Dense(len(train_data_all_10_percent.class_names), activation="softmax", name="output_layer")(x) # same number of outputs as classes
model = tf.keras.Model(inputs, outputs)

In [17]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 global_avg_pool_layer (Glo  (None, 1280)              0         
 balAveragePooling2D)                                            
                                                                 
 output_layer (Dense)        (None, 101)               129381    
                                                             

In [28]:
model.compile(loss="categorical_crossentropy",
             optimizer=tf.keras.optimizers.Adam(),
             metrics=["accuracy"])

In [29]:
# Compile
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(), # use Adam with default settings
              metrics=["accuracy"])

# Fit
history_all_classes_10_percent = model.fit(train_data_all_10_percent,
                                           epochs=5, # fit for 5 epochs to keep experiments quick
                                           validation_data=test_data,
                                           validation_steps=int(0.15 * len(test_data)), # evaluate on smaller portion of test data
                                           callbacks=[checkpoint_callback])

Epoch 1/5
237/237 [==============================] - 446s 2s/step - loss: 3.3681 - accuracy: 0.2712 - val_loss: 2.5410 - val_accuracy: 0.4206
Epoch 2/5
237/237 [==============================] - 443s 2s/step - loss: 2.1966 - accuracy: 0.4973 - val_loss: 2.0477 - val_accuracy: 0.4976
Epoch 3/5
237/237 [==============================] - 461s 2s/step - loss: 1.8116 - accuracy: 0.5716 - val_loss: 1.9182 - val_accuracy: 0.5124
Epoch 4/5
237/237 [==============================] - 492s 2s/step - loss: 1.6044 - accuracy: 0.6074 - val_loss: 1.8277 - val_accuracy: 0.5273
Epoch 5/5
237/237 [==============================] - 476s 2s/step - loss: 1.4455 - accuracy: 0.6453 - val_loss: 1.7752 - val_accuracy: 0.5347


In [ ]:
fine_tuning_results = model.evaluate(test_data)
fine_tuning_results

In [35]:
import pickle
from tensorflow.keras.models import load_model

with open('history_all_classes_10_percent.pkl', 'wb') as file:
    pickle.dump(history_all_classes_10_percent.history, file)

# Save Model
model.save('model.keras')

# Load Model (if needed)
loaded_model = load_model('model.keras')